In [8]:
# imports...
from datetime import datetime, time
import simpy
import random
import pandas as pd
from collections import defaultdict

# ----- VENUE CLASS -----

class Venue:
    def __init__(self, env, name, category, capacity):
        self.capacity = capacity
        self.env = env
        self.name = name
        self.category = category
        self.schedule = defaultdict(list)
        self.lock = simpy.Resource(env, capacity=1)

    def is_available(self, date, time_slot):
        try:
            start_new, end_new = [datetime.strptime(t, "%H:%M").time() for t in time_slot.split("-")]
        except Exception as e:
            print(f"Error parsing time slot {time_slot}: {e}")
            return False

        if date not in self.schedule:
            return True

        for booked in self.schedule[date]:
            start_booked, end_booked = [datetime.strptime(t, "%H:%M").time() for t in booked.split("-")]
            if start_new < end_booked and end_new > start_booked:
                return False
        return True

    def add_booking(self, date_str, time_slot):
        if self.is_available(date_str, time_slot):
            self.schedule[date_str].append(time_slot)
            return True
        return False


# ----- ADMIN CLASS -----
class Admin:
    def approve(self, request_info):
        response = input(f"Approve booking for {request_info['club']} at {request_info['venue']}? (y/n): ").lower()
        # send_email(f"Booking request from {request_info['club']} for {request_info['venue']} on {request_info['date']} at {request_info['time_slot']}")
        return response == 'y'

# ----- SYSTEM CLASS -----
class BookingSystem:
    def __init__(self, env):
        self.env = env
        self.venues = []
        self.admin = Admin()
        self.logs = []
        self.categories = defaultdict(list)  # key: category, value: list of venues

    def add_venue(self, venue):
        self.venues.append(venue)
        self.categories[venue.category].append(venue)

    def suggest_available_venues(self, date, time_slot):
        """Suggest venues available for a given date and time slot across all categories."""
        available_venues = {}
        for category, venues in self.categories.items():
            available_venues[category] = []
            for venue in venues:
                if venue.is_available(date, time_slot):
                    available_venues[category].append(venue.name)
        return available_venues

    def process_booking(self, club_name, venue_name, day, date_str, time_slot):
        yield self.env.timeout(0)
        venue = next((v for v in self.venues if v.name == venue_name), None)
        if venue:
            with venue.lock.request() as req:
                yield req
                # Now checking availability inside the lock
                if venue.add_booking(date_str, time_slot):
                    approved = self.admin.approve({'club': club_name, 'venue': venue.name, 'date': date_str, 'time_slot': time_slot})
                    if approved:
                        self.logs.append((self.env.now, club_name, venue_name, 'Approved', day, date_str, time_slot))
                        print(f"[{self.env.now}] ✅ {club_name} booked {venue_name} on {date_str} for {time_slot}")
                    else:
                        # If not approved by admin, roll back the booking
                        venue.schedule[date_str].remove(time_slot)
                        self.logs.append((self.env.now, club_name, venue_name, 'Rejected (Admin)', day, date_str, time_slot))
                        print(f"[{self.env.now}] ❌ Admin rejected booking for {club_name} at {venue_name}")
                else:
                    self.logs.append((self.env.now, club_name, venue_name, 'Rejected (Conflict)', day, date_str, time_slot))
                    print(f"[{self.env.now}] ❌ Booking conflict: {venue_name} is already booked on {date_str} for {time_slot}")



    def report(self):
        df = pd.DataFrame(self.logs, columns=['Time', 'Club', 'Venue', 'Status', 'Day', 'Date', 'Time Slot'])
        print("\n--- Booking Summary ---")
        print(df)
        import os
        file_path = "booking_log.csv"
        # Check if the file exists and is nonempty
        if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
            df.to_csv(file_path, mode='w', header=True, index=False)
        else:
            df.to_csv(file_path, mode='a', header=False, index=False)

# def send_email(body):
#     msg = EmailMessage()
#     msg.set_content(body)
#     msg['Subject'] = f'Approval for venue booking'
#     msg['From'] = '231030192@juitsolan.in'
#     msg['To'] = '231030195@juitsolan.in'

#     s = smtplib.SMTP('localhost')
#     s.send_message(msg)
#     s.quit()

# In this funciton, we validate input and then use the provided date and time slot to check bookings.
def collect_booking_requests(env, system):
    booking_requests = []
    n = int(input("Enter number of booking requests: "))
    for _ in range(n):
        club_name = input("\nEnter club name: ")
        day = input("Enter day of event (e.g. Monday): ")
        
        while True:
            date_input = input("Enter date of event (YYYY-MM-DD): ")
            try:
                datetime.strptime(date_input, "%Y-%m-%d")
                break
            except ValueError:
                print("Invalid date format. Please enter in YYYY-MM-DD format.")
        
        import re
        while True:
            time_slot = input("Enter time slot (e.g. 09:00-11:00): ")
            if re.match(r'^\d{2}:\d{2}-\d{2}:\d{2}$', time_slot):
                try:
                    start, end = time_slot.split('-')
                    datetime.strptime(start, "%H:%M")
                    datetime.strptime(end, "%H:%M")
                    break
                except ValueError:
                    print("Invalid time format.")
            else:
                print("Format should be HH:MM-HH:MM.")

        available_venues = system.suggest_available_venues(date_input, time_slot)
        if not any(available_venues.values()):
            print("No venues available for the selected date and time slot.")
            continue

        print("\nAvailable venues:")
        for category, venues in available_venues.items():
            if venues:
                print(f"{category}: {', '.join(venues)}")

        while True:
            venue_name = input("Select a venue from above list: ")
            valid_venues = [v for sub in available_venues.values() for v in sub]
            if venue_name not in valid_venues:
                print("Invalid venue. Try again.")
                continue
            else:
                break

        booking_requests.append((club_name, venue_name, day, date_input, time_slot))
    
    return booking_requests


# ----- SETUP SIMULATION -----
def setup_simulation():
    env = simpy.Environment()
    system = BookingSystem(env)

    # Adding venues
    system.add_venue(Venue(env, "Auditorium", "Auditorium", 1000))
    system.add_venue(Venue(env, "LT-1", "Lecture Theatre", 300))
    system.add_venue(Venue(env, "CR-1", "Conference Room", 100))

    # Collect all booking requests before running the simulation
    booking_requests = collect_booking_requests(env, system)
    
    for booking in booking_requests:
        env.process(system.process_booking(*booking))

    env.run()
    system.report()


# Run the simulation
if __name__ == '__main__':
    setup_simulation()


Available venues:
Auditorium: Auditorium
Lecture Theatre: LT-1
Conference Room: CR-1

Available venues:
Auditorium: Auditorium
Lecture Theatre: LT-1
Conference Room: CR-1

Available venues:
Auditorium: Auditorium
Lecture Theatre: LT-1
Conference Room: CR-1
[0] ✅ acm booked LT-1 on 2025-05-04 for 09:00-13:30
[0] ❌ Booking conflict: LT-1 is already booked on 2025-05-04 for 09:00-11:00
[0] ❌ Booking conflict: LT-1 is already booked on 2025-05-04 for 09:00-10:00

--- Booking Summary ---
   Time  Club Venue               Status     Day        Date    Time Slot
0     0   acm  LT-1             Approved  monday  2025-05-04  09:00-13:30
1     0   acm  LT-1  Rejected (Conflict)  monday  2025-05-04  09:00-11:00
2     0  ieee  LT-1  Rejected (Conflict)  monday  2025-05-04  09:00-10:00
